# Exemple d'utilisation d'un transfcormer dans le cadre de la génération de code.

On utilise au chois les modeles

* codegen-350M-mono
* codegen-2B-mono
*

In [114]:
model_name = "Salesforce/codegen-350M-mono"

In [115]:
# Parametres du model

max_length=200,  # Adjust as needed
num_beams=5,      # Adjust for quality/speed trade-off
temperature=0.7,  # Adjust for creativity (higher = more creative)
top_k=40,         # Adjust for sampling
top_p=0.95,        # Adjust for sampling
pad_token_id=tokenizer.eos_token_id # Important for some models

# Observations

* En promptant sur un debut de code le model est capable de prédire du code correcte sur des soucis simple mais pas parfait
* En utilisant le model il semble y avoir conservation du context
    * Pour vérifier réinitialisé l'appel entre chaque

## Initialisation du model

In [116]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Création du tokenniser
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Chargement du model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Vérifie si le GPU est disponible avant de continuer
if torch.cuda.is_available():
    # Sélectionne spécifiquement le premier GPU (device 0)
    device = torch.device("cuda:0")
    print("Utilisation du GPU :", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU non trouvé, utilisation du CPU.")

# Déplace le modèle vers le GPU 0
# ca plante !
# model.to(device)


Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Utilisation du GPU : NVIDIA GeForce RTX 3070 Laptop GPU


In [117]:
def do_predict(model,
               prompt: str,
               max_length: int = 200,
               temperature: float = 0.7,
               top_k: int = 40,
               top_p: float = 0.9,
               pad_token_id: int = tokenizer.eos_token_id):
    print(f"Prompte : {prompt}\n----")
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    print(f"Input ids : {input_ids}")
    print("---")
    generated_ids = model.generate(input_ids,
                                   max_length=max_length,
                                   temperature=temperature,
                                   top_k=top_k,
                                   top_p=top_p,
                                   pad_token_id=pad_token_id,
                                   do_sample=True)
    print("----")
    print(generated_ids[0])
    print("----")
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)


## Utilisation pour faire de la completion de code

On fournis un fragment de code et on demande au modele de compléter.

In [118]:
text = """
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:
"""

print(do_predict(model, text))


Prompte : 
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:

----
Input ids : tensor([[  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
           257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
            90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
            65, 36786,   198, 17772,    25,   198]])
---
----
tensor([  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
          257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
           90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
           65, 36786,   198, 17772,    25,   198, 50284,  4798,     7,    69,
            1,    90,    65,    92,  1556,  7418,  2634,  5034,   333, 28141,
         1391,    64,    92,  4943,   198,   198,     2,  1475,  2798,   501,
          198,    64,   796,   493,     7, 15414,  7203,    64,   796,   366,
         4008,   198,    65,   796,   493,     7, 15414,  720

In [119]:
print(do_predict(model, text, temperature=1))

Prompte : 
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:

----
Input ids : tensor([[  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
           257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
            90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
            65, 36786,   198, 17772,    25,   198]])
---
----
tensor([  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
          257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
           90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
           65, 36786,   198, 17772,    25,   198, 50284,  4798,     7,    69,
            1,    90,    65,    92,  1556,  1167,  2634,  5034,   333, 28141,
         1391,    64,    92,  4943,   198,   198,     2,  9724,   291,  1538,
          274,   198,   198,   496,   796,  1160,   198,   361,  2479, 18189,
         1248,    25,   198, 50284,  4798,  7203,    34,  585

In [120]:
print(do_predict(model, text, temperature=0.01))

Prompte : 
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:

----
Input ids : tensor([[  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
           257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
            90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
            65, 36786,   198, 17772,    25,   198]])
---
----
tensor([  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
          257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
           90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
           65, 36786,   198, 17772,    25,   198, 50284,  4798,     7,    69,
            1,    90,    65,    92,  1556,  7418,  2634,  5034,   333, 28141,
         1391,    64,    92,  4943,   198,   198,     2,  1475,  2798,   501,
          198,   198,     2,   352,    13,   198,     2,  1550,  3512,    68,
        28141,   300,     6, 22602,   271, 15093,   390,   47

In [121]:
print(do_predict(model, text, temperature=0.01, max_length=1024))

Prompte : 
a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:

----
Input ids : tensor([[  198,    64,   796,   352,   198,    65,   796,   362,   198,   361,
           257,  1875,   275,    25,   198, 50284,  4798,     7,    69,     1,
            90,    64,    92,  1556,  7418,  2634,  5034,   333, 28141,  1391,
            65, 36786,   198, 17772,    25,   198]])
---
----
tensor([198,  64, 796,  ..., 959,  13, 198])
----

a = 1
b = 2
if a > b:
    print(f"{a} est supérieur à {b}"
else:
    print(f"{b} est supérieur à {a}")

# Exercice

# 1.
# On demande à l'utilisateur de saisir un nombre entier.
# On affiche le nombre entier saisi.
# On demande à l'utilisateur de saisir un nombre entier.
# On affiche le nombre entier saisi.
# On demande à l'utilisateur de saisir un nombre entier.
# On affiche le nombre entier saisi.
# On demande à l'utilisateur de saisir un nombre entier.
# On affiche le nombre entier saisi.
# On demande à l'utilisateur de saisir un nombre entier.
# 

In [122]:
model = AutoModelForCausalLM.from_pretrained(model_name)
print(do_predict(model, """
generete une fonction pour calculer la somme des 5 premiers chiffres
"""))

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

Prompte : 
generete une fonction pour calculer la somme des 5 premiers chiffres

----
Input ids : tensor([[  198,  5235,   567,   660, 17809,   277,   261,   596, 12797,  5204,
           263,  8591,  3870,  1326,   748,   642,  4199,  3183,   442,   733,
           411,   198]])
---
----
tensor([  198,  5235,   567,   660, 17809,   277,   261,   596, 12797,  5204,
          263,  8591,  3870,  1326,   748,   642,  4199,  3183,   442,   733,
          411,   198,     7,   785,  1326,  1058,   657,    11,   352,    11,
          362,    11,   513,    11,   604,    11,   642,     8,   198, 37811,
          628,   198,  4299,  3870,  1326,    62,   354,   733,   411,     7,
           77,  2599,   198, 50284, 37811,   198, 50284,    25, 17143,   299,
           25,   299,  2381,   260,   390,   442,   733,   411,   198, 50284,
           25,  7783,    25,  8591,  3870,  1326,   748,   642,  4199,  3183,
          442,   733,   411,   198, 50284, 37811,   198, 50284,  7783,  2160,
        

## test du code

1er run

In [123]:
def somme_chiffres(n):
    """
    :param n: nombre de chiffres
    :return: la somme des 5 premiers chiffres
    """
    somme = 0
    for i in range(n):
        somme += i
    return somme

def main():
    """
    :return: la somme des 5 premiers chiffres
    """
    n = int(input("Entrer un nombre : "))
    print(somme_chiffres(n))

if __name__ == "__main__":
    main()

KeyboardInterrupt: Interrupted by user

2eme run

In [101]:
### j'ai ajouté le """ pour rendre le code valide
"""
generete une fonction pour calculer la somme des 5 premiers chiffres
définit les chiffres en fonction de la division par 5
"""

def somme():

    somme = 0

    for i in range(5):
        somme = somme + int(input("Entrez le chiffre : "))

    print(somme)

somme()


5
